In [107]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor

In [108]:
Nimply = namedtuple("Nimply", "row, num_objects")
Individual = namedtuple("Individual", ["genome", "fitness"])

POPULATION_SIZE = 20      
OFFSPRING_SIZE = 20        
NUM_GENERATIONS = 5      
PROBLEM_SIZE = 5
TOURNAMENT_SIZE =2
GENETIC_OPERATOR_RANDOMNESS = 0.3
MUTATION_RANDOMNESS = 0.5
CROSSOVER_RANDOMNESS = 0.5
num_rows=5

In [109]:
class Nim:
    def __init__(self, num_rows: int, k: int = None):
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self):
        return tuple(self._rows)

    @property
    def k(self):
        return self._k

    #toglie num oggetti dalla row scelta
    def nimming(self, ply: Nimply) -> None: 
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [110]:
##ritorna 0 solo se è possibile fare una mossa sicura
def nim_sum(state: Nim):
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim):
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state) 

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [111]:
#strategie dell'avversario

def optimal_startegy(state: Nim):
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

def dumb_PCI(state: Nim):
    """Pick always the minimum(maximum) possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], -m[1])))

In [112]:
#EVOLUTIONARY RULES

def tournament(population, tournament_size=TOURNAMENT_SIZE):          
    return min(random.choices(population, k=tournament_size), key=lambda i: i.fitness) 

def cross_over(g1, g2):                      
    if random.random() < CROSSOVER_RANDOMNESS:
        return (g1[0], g2[1])
    else:
        return(g2[0], g1[1])

def mutation_rows(row):
    old_row = row
    new_row = 0
    if random.random() < MUTATION_RANDOMNESS: #+
        if old_row + 1 >= num_rows:
            new_row = old_row - 1
        else: 
            new_row = old_row + 1
    else: #-
        if old_row - 1 < 0:
            new_row = old_row + 1
        else:
            new_row = old_row - 1
    return new_row

def mutation_obj(obj, new_row, nim):
    old_obj = obj
    new_obj = 1
    if random.random() < MUTATION_RANDOMNESS: #+
        if old_obj + 1 <= nim.rows[new_row] and old_obj >= 2:
            new_obj = old_obj - 1
        else: 
            new_obj = old_obj + 1
    else: #-
        if old_obj - 1 <= 0:
            new_obj = old_obj + 1
        else:
            new_obj = old_obj - 1
    return new_obj

def mutation(g, nim):                                
    new_row = mutation_rows(g[0])
    new_obj = mutation_obj(g[1], new_row, nim)
    return (new_row, new_obj)

def compute_fitness(genome, nim): 
    tmp = deepcopy(nim)
    tupla = (genome[0], genome[1])
    if nim._rows[genome[0]] < genome[1]:
        return 100
    tmp.nimming(tupla)
    return nim_sum(tmp)

In [113]:

def best_move(nim):
    global population
    for g in range(NUM_GENERATIONS):
        offspring = list()
        for i in range(OFFSPRING_SIZE):
            if random.random() < GENETIC_OPERATOR_RANDOMNESS:                         
                p = tournament(population)                  
                o = mutation(p.genome, nim)                    
            else:                                          
                p1 = tournament(population)                 
                p2 = tournament(population)
                o = cross_over(p1.genome, p2.genome)            
            f = compute_fitness(o, nim)                                                          
            offspring.append(Individual(o, f))                 
        population += offspring
        population = set(population)  #remove duplicate
        population = list(population)  
        population = sorted(population, key=lambda i: i[1], reverse = False)[:POPULATION_SIZE]
    
    return population[0][0]

In [114]:
logging.getLogger().setLevel(logging.DEBUG)

strategy = (best_move, dumb_PCI)

nim = Nim(num_rows)
population = list()
while len(population)<POPULATION_SIZE:
    row = random.randint(0, num_rows-1)
    obj = random.randint(1, nim.rows[row])    
    genome = (row,obj)
    population.append(Individual(genome, compute_fitness(genome, nim))) 

logging.debug(f"status: Initial board  -> {nim}")
player = 0
#0 => agent, 1 => avversario
while nim:
    
    if player == 0:
        ply = strategy[player](nim)
        print(ply)
        while nim._rows[ply[0]] < ply[1]:
            population.pop(0)
            ply = population[0][0]
            print(ply)
    else:
        ply = strategy[player](nim)
        
    nim.nimming(ply)  
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")


DEBUG:root:status: Initial board  -> <1 3 5 7 9>
DEBUG:root:status: After player 0 -> <1 3 5 7 1>
DEBUG:root:status: After player 1 -> <0 3 5 7 1>
DEBUG:root:status: After player 0 -> <0 3 5 7 0>
DEBUG:root:status: After player 1 -> <0 2 5 7 0>
DEBUG:root:status: After player 0 -> <0 2 4 7 0>
DEBUG:root:status: After player 1 -> <0 1 4 7 0>
DEBUG:root:status: After player 0 -> <0 1 4 5 0>
DEBUG:root:status: After player 1 -> <0 0 4 5 0>
DEBUG:root:status: After player 0 -> <0 0 4 3 0>
DEBUG:root:status: After player 1 -> <0 0 3 3 0>
DEBUG:root:status: After player 0 -> <0 0 3 1 0>
DEBUG:root:status: After player 1 -> <0 0 2 1 0>
DEBUG:root:status: After player 0 -> <0 0 1 1 0>
DEBUG:root:status: After player 1 -> <0 0 0 1 0>
DEBUG:root:status: After player 0 -> <0 0 0 0 0>
INFO:root:status: Player 0 won!


(4, 8)
(4, 1)
(4, 1)
(2, 1)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
(2, 1)
(2, 1)
(3, 1)
